<a href="https://colab.research.google.com/github/SPLV2024/MLMODEL/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pandas openpyxl transformers


In [ ]:
import pandas as pd
from transformers import pipeline

def load_workbook(file_path):
    """Load the Excel workbook and return a dictionary of DataFrames keyed by sheet names."""
    excel_data = pd.read_excel(file_path, sheet_name=None)
    return excel_data

def search_data_with_nlp(dictionary, description, nlp_model):
    """
    Search for table names and column descriptions within the workbook dictionary based on a description using NLP.

    Args:
        dictionary (dict): Dictionary of DataFrames representing the workbook.
        description (str): Description provided by the client.
        nlp_model: Pre-trained NLP model for text classification.

    Returns:
        pd.DataFrame: DataFrame with columns: Sheet, Table Name, Column Name, Description.
    """
    results = []
    for sheet, df in dictionary.items():
        df = df.astype(str)
        for idx, row in df.iterrows():
            row_text = ' '.join(row.values)
            input_text = f"{description} {row_text}"
            prediction = nlp_model(input_text)
            if prediction[0]['score'] > 0.5:  # Assuming a threshold for relevance
                results.append({
                    'Sheet': sheet,
                    'Table Name': row.get('Table Name', 'N/A'),
                    'Column Name': row.get('Column Name', 'N/A'),
                    'Description': row.get('Description', 'N/A')
                })

    return pd.DataFrame(results)

def display_results(results):
    """Display the search results."""
    print(results)


In [20]:
# Path to your Excel file
file_path = '/content/drive/MyDrive/Course Structure for Analytics Boot camp.xlsx'



# Load the workbook
workbook_data = load_workbook(file_path)

# Description provided by the client
description = 'Cleansing, Transforming and Shaping Data'  # Replace with the description provided by the client

# Load pre-trained NLP model
nlp_model = pipeline('text-classification', model='facebook/bart-large-mnli')

# Get search results
search_results = search_data_with_nlp(workbook_data, description, nlp_model)

# Display the results
display_results(search_results)

        Sheet Table Name Column Name Description
0    Summary         N/A         N/A         N/A
1    Summary         N/A         N/A         N/A
2    Summary         N/A         N/A         N/A
3    Summary         N/A         N/A         N/A
4    Summary         N/A         N/A         N/A
..        ...        ...         ...         ...
274        ML        N/A         N/A         N/A
275        ML        N/A         N/A         N/A
276        ML        N/A         N/A         N/A
277        ML        N/A         N/A         N/A
278        ML        N/A         N/A         N/A

[279 rows x 4 columns]


In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_workbook(file_path):
    """Load the Excel workbook and return a dictionary of DataFrames keyed by sheet names."""
    excel_data = pd.read_excel(file_path, sheet_name=None)
    return excel_data

def prepare_corpus(dictionary):
    """Prepare a corpus from the workbook dictionary."""
    corpus = []
    row_metadata = []
    for sheet, df in dictionary.items():
        df = df.astype(str)
        for idx, row in df.iterrows():
            for col in df.columns:
                cell_text = row[col]
                corpus.append(cell_text)
                row_metadata.append({
                    'Sheet': sheet,
                    'Table Name': row.get('S.No', 'N/A'),
                    'Column Name': row.get('TOPIC', 'N/A'),
                    'Description': row.get('SUB TOPICS COVERED', 'N/A'),
                    'Cell Text': cell_text
                })
    return corpus, row_metadata

def search_data(description, corpus, row_metadata, vectorizer, threshold=0.2):
    """
    Search for relevant rows based on a description using TF-IDF and cosine similarity.

    Args:
        description (str): Description provided by the client.
        corpus (list): List of documents.
        row_metadata (list): Metadata for each row.
        vectorizer: TF-IDF vectorizer.
        threshold (float): Similarity threshold for filtering results.

    Returns:
        pd.DataFrame: DataFrame with columns: Sheet, Table Name, Column Name, Description.
    """
    description_vec = vectorizer.transform([description])
    corpus_vec = vectorizer.transform(corpus)
    similarities = cosine_similarity(description_vec, corpus_vec).flatten()

    results = []
    for idx, similarity in enumerate(similarities):
        if similarity >= threshold:
            results.append(row_metadata[idx])

    return pd.DataFrame(results)

def display_results(results):
    """Display the search results as a DataFrame."""
    print(results)

# Path to your Excel file
file_path = '/content/drive/MyDrive/Course Structure for Analytics Boot camp.xlsx'

# Load the workbook
workbook_data = load_workbook(file_path)

# Prepare the corpus and metadata
corpus, row_metadata = prepare_corpus(workbook_data)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

# Description provided by the client
description = 'tables managing column sorting'  # Replace with the description provided by the client

# Get search results
search_results = search_data(description, corpus, row_metadata, vectorizer)

# Display the results as a DataFrame
search_results


,Sheet,Table Name,Column Name,Description,Cell Text
0,SQL,N/A,N/A,N/A,Working with Column alias
1,SQL,N/A,N/A,N/A,Data filtering and sorting with in single table
2,SQL,N/A,N/A,N/A,Multi column sub queries
3,Excel,N/A,N/A,N/A,Sorting and Filtering
4,Data Visualization,2.0,"Cleansing, Transforming and Shaping Data",N/A,Sorting and filtering
5,Data Visualization,nan,nan,N/A,Managing columns
6,Data Visualization,nan,nan,N/A,Using column transformations
7,Data Visualization,3.0,Designing a Data Model,N/A,Define the tables
